In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../..")

from heritageconnector import datastore
from heritageconnector.utils.wikidata import qid_to_url

import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

from IPython.display import display

from tqdm import tqdm

## 1. create table of links betweeen SMG and Wikidata people

In [2]:
df = pd.read_pickle("../../GITIGNORE_DATA/filtering_people_orgs_result.pkl")

display(df['qcodes_filtered'].apply(len).value_counts())
df_links = df[df['qcodes_filtered'].apply(len) == 1]

df_links['QID'] = df_links['qcodes_filtered'].apply(lambda i: i[0])

df_links = df_links[["LINK_ID", "QID"]]

# transform IDs to URLs
df_links["LINK_ID"] = df_links["LINK_ID"].apply(lambda i: f"https://collection.sciencemuseumgroup.org.uk/people/cp{i}")
df_links["QID"] = df_links["QID"].apply(qid_to_url)

df_links.head()

0    14375
1     3714
2        6
Name: qcodes_filtered, dtype: int64

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,LINK_ID,QID
3,https://collection.sciencemuseumgroup.org.uk/people/cp1027,http://www.wikidata.org/entity/Q2007300
10,https://collection.sciencemuseumgroup.org.uk/people/cp10203,http://www.wikidata.org/entity/Q90261
23,https://collection.sciencemuseumgroup.org.uk/people/cp10503,http://www.wikidata.org/entity/Q18759224
42,https://collection.sciencemuseumgroup.org.uk/people/cp11221,http://www.wikidata.org/entity/Q310833
57,https://collection.sciencemuseumgroup.org.uk/people/cp11809,http://www.wikidata.org/entity/Q7341431


## 2. Add links to ES index

In [4]:
for _, row in tqdm(df_links.iterrows(), total=len(df_links)):
    datastore.add_same_as(row["LINK_ID"], row["QID"])

100%|██████████| 3714/3714 [02:19<00:00, 26.55it/s]


In [5]:
datastore.get_by_uri(df_links.iloc[0]["LINK_ID"])["_source"]["graph"]

{'@context': [{'@foaf': 'http://xmlns.com/foaf/0.1/', '@language': 'en'},
  {'@language': 'en', '@schema': 'http://www.w3.org/2001/XMLSchema#'},
  {'@language': 'en', '@owl': 'http://www.w3.org/2002/07/owl#'}],
 '@foaf:givenName': {'@value': 'Link, O Winston'},
 '@id': 'https://collection.sciencemuseumgroup.org.uk/people/cp1027',
 '@owl:sameAs': {'@id': 'http://www.wikidata.org/entity/Q2007300'},
 '@schema:birthPlace': {'@value': 'Brooklyn, New York city, New York state, United States'},
 '@schema:deathPlace': {'@value': 'South Salem, Westchester county, New York state, United States'},
 '@schema:disambiguatingDescription': {'@value': 'Y'},
 '@schema:gender': {'@value': 'Male'},
 '@schema:occupation': {'@value': 'photographer'}}